# How to read airfoil shapes data (already reparametrized) 

## How to read airfoils data from .npz file

In [1]:
# import matplotlib.pyplot as plt


# from g2aero.Grassmann import *
# from g2aero.utils import *
#  Dataset

In [6]:
import os
import numpy as np
shapes_folder  = os.path.join(os.getcwd(), '../../data/airfoils/', )

shapes = np.load(os.path.join(shapes_folder, 'shapes.npz'))['shapes']
# classes= np.load(os.path.join(shapes_folder, 'shapes.npz'))['classes']
print("Dataset:")
print(f"Shape of data = {shapes.shape}")
print(f"N shapes = {shapes.shape[0]}")
print(f"n landmarks in every shape = {shapes.shape[1]}")

Dataset:
Shape of data = (2588, 500, 2)
N shapes = 2588
n landmarks in every shape = 500


## How to construct PGA space and get coordinates 

In [7]:
from g2aero.perturbation import PGAspace
pga, t = PGAspace.create_from_dataset(shapes)

Karcher mean convergence:
||V||_F = 0.45130427482058105
||V||_F = 0.002084258010036918
||V||_F = 1.5924782056615687e-05
||V||_F = 1.8980134151380567e-07
||V||_F = 2.4778808741590328e-09


In [ ]:
import pandas as pd
import seaborn as sns
coord_names = ['$t_1$', '$t_2$', '$t_3$', '$t_4$']
df_all = pd.DataFrame(data=t[:, :4], columns=coord_names)
sns_plot = sns.pairplot(df_all, x_vars=coord_names, y_vars=coord_names,
                        diag_kind='kde', palette='Dark2', corner=True, plot_kws=dict(s=15))

## Reconstruction error

In [ ]:
def norm_inf2(sh1, sh2):
    d = np.max(np.linalg.norm(sh1-sh2, ord=2, axis=1))
    return d

def norm_te(sh1, sh2):
    return np.linalg.norm(sh1[(0, -1)] - sh2[(0,-1)], ord=2, axis=1)

r_arr = np.arange(2, 2*n_landmarks)
err_gr = np.zeros((data.n_shapes, len(r_arr)))
err_te = np.zeros((data.n_shapes, len(r_arr)))
err_inf2 = np.zeros((data.n_shapes, len(r_arr)))
for i, r in enumerate(r_arr):
    for j, coord in enumerate(t[:, :r]): 
        shape_new = perturb_gr_shape(pga.Vh, pga.karcher_mean, coord)
        R = procrustes(shape_new, data.shapes_gr[j])
        shape_new = shape_new @ R
        err_gr[j, i] = distance(shape_new, data.shapes_gr[j])
        phys_shape = shape_new @ data.M[j] + data.b[j]
        err_inf2[j, i] = norm_inf2(phys_shape, shapes[j])
        err_te[j, i] = norm_inf2(phys_shape, shapes[j])